In [1]:
import numpy as np
from matplotlib import pyplot as plt
from numpy import random as rnd
import os,warnings,sys,itertools,datetime,time,timeit

import torch
from torch import nn,optim
import torch.functional as F

import re
import unicodedata

import pandas as pd

import _pickle as pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def generate_lang_models(readpath,storepath):
    with open(readpath+'europarl-v7.de-en.en','r',encoding='utf-8') as file:
        data_en = file.read()
    with open(readpath+'europarl-v7.de-en.de','r',encoding='utf-8') as file:
        data_de = file.read()
    df = pd.DataFrame(data_en.split('\n'),data_de.split('\n')).reset_index(drop=False).rename(columns={
        'index':'DE',
        0:'EN'
    })
    input_lang = Lang('de')
    output_lang = Lang('en')

    for r,cs in df.iterrows():
        input_lang.addSentence(normalizeString(cs.DE))
        output_lang.addSentence(normalizeString(cs.EN))

    df.to_pickle(storepath + 'pairs.pkl')
    with open(storepath + 'input_lang_model.pkl','wb') as f:
        pickle.dump(input_lang,f)
    with open(storepath + 'output_lang_model.pkl','wb') as f:
        pickle.dump(output_lang,f)
    return

In [3]:
class Lang:
    def __init__(self,name):
        self.name = name
        self.word2index = {}
        self.index2word = {0:'SOS',1:'EOS'}
        self.word2count = {}
        self.nwords = 2
    def addWord(self,word):
        if word not in self.word2index:
            self.word2index[word] = self.nwords
            self.index2word[self.nwords] = word
            self.word2count[word] = 1
            self.nwords += 1
        else:
            self.word2count[word] += 1
    def addSentence(self,sent):
        for word in sent.split(' '):
            self.addWord(word)

In [4]:
def normalizeString(s):
    s = s.lower().strip()
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
    s = re.sub(r'([.!?])',r' \1',s)
    s = re.sub(r'[^a-zA-Z.!?]+', r' ',s)
    return s

In [5]:
root_path = './../../../TF_Data/Dropbox/TF_data/SEQ2SEQ/'
#root_path = '../../../../../../../TF_data/SEQ2SEQ/''

with open(root_path + 'input_lang_model.pkl','rb') as f:
    input_lang = pickle.load(f)
with open(root_path + 'output_lang_model.pkl','rb') as f:
    output_lang = pickle.load(f)
with open(root_path + 'pairs.pkl','rb') as f:
    df = pickle.load(f)

In [6]:
df

,DE,EN
0,Wiederaufnahme der Sitzungsperiode,Resumption of the session
1,"Ich erkläre die am Freitag, dem 17. Dezember u...",I declare resumed the session of the European ...
2,"Wie Sie feststellen konnten, ist der gefürchte...","Although, as you will have seen, the dreaded '..."
3,Im Parlament besteht der Wunsch nach einer Aus...,You have requested a debate on this subject in...
4,Heute möchte ich Sie bitten - das ist auch der...,"In the meantime, I should like to observe a mi..."
...,...,...
1920205,"Ich will die Aussprache über ""Millennium ja, M...",I am not going to re-open the 'Millennium or n...
1920206,Unterbrechung der Sitzungsperiode,Adjournment of the session
1920207,Ich erkläre die Sitzungsperiode des Europäisch...,I declare the session of the European Parliame...
1920208,(Die Sitzung wird um 10.50 Uhr geschlossen.),(The sitting was closed at 10.50 a.m.)
